In [1]:
# 1. Install the libraries
!pip install -q "transformers<4.41" gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 46.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.2 which is incompatible.


In [2]:
# 2. Set up the chatbot pipeline
from transformers import pipeline, Conversation
import gradio as gr

MODEL_NAME = "facebook/blenderbot-400M-distill"
chatbot = pipeline("conversational", model=MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [3]:
# 3. Quick console check (optional)
conversation = Conversation("Hi, how are you doing today?")
conversation = chatbot(conversation)
print("=== Console demo ===")
print(conversation)
print()

# Add another turn
conversation.add_user_input("What do you like to do in your free time?")
conversation = chatbot(conversation)
print(conversation)
print()


No chat template is defined for this tokenizer - using the default template for the BlenderbotTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



=== Console demo ===
Conversation id: 1f46c3e5-87aa-403c-906f-c7dc4f45b2fc
user: Hi, how are you doing today?
assistant:  I'm doing well, thank you. I hope you are as well. How are you?


Conversation id: 1f46c3e5-87aa-403c-906f-c7dc4f45b2fc
user: Hi, how are you doing today?
assistant:  I'm doing well, thank you. I hope you are as well. How are you?
user: What do you like to do in your free time?
assistant:  I like to spend time with my family and friends. What about you? Do you have any hobbies?




In [ ]:
# 4. Gradio interface
message_list = []
response_list = []

def mini_chatbot(message, history):
    """
    Gradio ChatInterface callback.
    `message`:  latest user input (str)
    `history`:  list of (user_msg, bot_msg) tuples so far
    """
    # Split the history into parallel lists
    past_user_inputs = [h[0] for h in history]
    generated_responses = [h[1] for h in history]

    # Build a Conversation object
    conversation = Conversation(
        text=message,
        past_user_inputs=past_user_inputs,
        generated_responses=generated_responses
    )

    # Generate the response
    conversation = chatbot(conversation)

    # The last response is what we return to Gradio
    return conversation.generated_responses[-1]

# Create and launch the interface
demo_chatbot = gr.ChatInterface(
    fn=mini_chatbot,
    title="🤖 BlenderBot-400M Mini Chat",
    description="A tiny chatbot powered by Facebook's BlenderBot-400M-distill. "
                "Type below and press Enter to start the conversation."
)

demo_chatbot.launch(debug=True)   # set share=True if running in Colab

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://34a37e5ee8f95cb368.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
